In [1]:
from astropy.time import Time, TimeDelta
import astropy.units as u
import glob
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import moviepy.editor as mpy
from moviepy.video.io.bindings import mplfig_to_npimage
import numpy as np
import os
import pandas as pd
import sunpy.coordinates.sun as sn
import scipy.ndimage as ndi
import scipy.stats as st
# Our own library for using spice with STEREO (https://github.com/LukeBarnard/stereo_spice)
from stereo_spice.coordinates import StereoSpice
# Local packages
import HUXt as H

spice = StereoSpice()


In [2]:
def setup_huxt(start_time, uniform_wind=True):
    """
    Initialise HUXt with some predetermined boundary/initial conditions
    start_time should be astropy.Time object.
    wind should be uniform or structured
    """
    cr_num = np.fix(sn.carrington_rotation_number(start_time))
    ert = H.Observer('EARTH', start_time)

    # Set up HUXt for a 5 day simulation with homogenous inner boundary.
    vr_in, br_in = H.Hin.get_MAS_long_profile(cr_num, ert.lat.to(u.deg))
    if uniform_wind:
        vr_in = np.zeros(vr_in.shape) + 400*vr_in.unit
        
    model = H.HUXt(v_boundary=vr_in, cr_num=cr_num, cr_lon_init=ert.lon_c, latitude=ert.lat.to(u.deg),
                   br_boundary=br_in, lon_start=270*u.deg, lon_stop=90*u.deg, simtime=3.5*u.day, dt_scale=4)
    
    return model

def get_base_cme(v=1000, lon=0, lat=0, width=45, thickness=5):
    """
    Return the base CME, which is used to establish the pseudo-truth CME and the SIR ensemble
    """
    t_launch = (1*u.hr).to(u.s)
    cme = H.ConeCME(t_launch=t_launch, longitude=lon*u.deg, latitude=lat*u.deg, width=width*u.deg, v=v*(u.km/u.s), thickness=thickness*u.solRad)
    return cme

def perturb_cone_cme(cme):
    """
    Perturb a ConeCME's parameters. Used to establish the pseudo-truth CME and the initial SIR ensemble members. 
    """
    lon_spread = 10*u.deg
    lat_spread = 10*u.deg
    width_spread = 10*u.deg
    v_spread = 100*(u.km/u.s)
    thickness_spread = 1*u.solRad
    
    randoms = np.random.uniform(-1,1,5)
    lon_new = cme.longitude + randoms[0]*lon_spread
    lat_new = cme.latitude + randoms[1]*lat_spread
    width_new = cme.width + randoms[2]*width_spread
    v_new = cme.v + randoms[3]*v_spread
    thickness_new = cme.thickness + randoms[4]*thickness_spread
    
    cme_perturb = H.ConeCME(t_launch=cme.t_launch,
                            longitude=lon_new,
                            latitude=lat_new,
                            width=width_new,
                            v=v_new,
                            thickness=thickness_new)
    return cme_perturb


class Observer:
    
    @u.quantity_input(longitude=u.deg)
    def __init__(self, model, longitude, el_min=4.0, el_max=30.0):
        
        ert_ephem = model.get_observer('EARTH')
        
        self.time = ert_ephem.time 
        self.r = ert_ephem.r
        self.lon = ert_ephem.lon + longitude
        self.lat = ert_ephem.lat
        self.el_min = el_min
        self.el_max = el_max
        # Force longitude into 0-360 domain
        id_over = self.lon > 360*u.deg
        id_under = self.lon < 0*u.deg
        if np.any(id_over):
            self.lon[id_over] = self.lon[id_over] - 360*u.deg
        if np.any(id_under):
            self.lon[id_under] = self.lon[id_under] + 360*u.deg
        
        cme = model.cmes[0]
        self.model_flank = self.compute_flank_profile(cme)
        
    def compute_flank_profile(self, cme):
        """
        Compute the time elongation profile of the flank of a ConeCME in HUXt. The observer longtidue is specified relative to Earth,
        and but otherwise matches Earth's coords. 

        Parameters
        ----------
        observer_lon: Angular separation of Earth and the observer, in HEEQ.
        cme: A ConeCME object from a completed HUXt run (i.e the ConeCME.coords dictionary has been populated).
        Returns
        -------
        obs_profile: Pandas dataframe giving the coordinates of the ConeCME flank from STA's perspective, including the
                    time, elongation, position angle, and HEEQ radius and longitude.
        """
        times = Time([coord['time'] for i, coord in cme.coords.items()])

        # Compute observers location using earth ephem, adding on observers longitude offset from Earth and correct for runover 2*pi
        flank = pd.DataFrame(index=np.arange(times.size), columns=['time', 'el', 'r', 'lon'])
        flank['time'] = times.jd

        for i, coord in cme.coords.items():

            if len(coord['r']) == 0:
                flank.loc[i, ['lon','r', 'el']] = np.NaN
                continue

            r_obs = self.r[i]
            x_obs = self.r[i] * np.cos(self.lat[i]) * np.cos(self.lon[i])
            y_obs = self.r[i] * np.cos(self.lat[i]) * np.sin(self.lon[i])
            z_obs = self.r[i] * np.sin(self.lat[i])

            lon_cme = coord['lon']
            lat_cme = coord['lat']
            r_cme = coord['r']

            x_cme = r_cme * np.cos(lat_cme) * np.cos(lon_cme)
            y_cme = r_cme * np.cos(lat_cme) * np.sin(lon_cme)
            z_cme = r_cme * np.sin(lat_cme)
            #############
            # Compute the observer CME distance, S, and elongation

            x_cme_s = x_cme - x_obs
            y_cme_s = y_cme - y_obs
            z_cme_s = z_cme - z_obs
            s = np.sqrt(x_cme_s**2 + y_cme_s**2 + z_cme_s**2)

            numer = (r_obs**2 + s**2 - r_cme**2).value
            denom = (2.0 * r_obs * s).value
            e_obs = np.arccos(numer / denom)

            # Find the flank coordinate and update output
            id_obs_flank = np.argmax(e_obs)       
            flank.loc[i, 'lon'] = lon_cme[id_obs_flank].value
            flank.loc[i, 'r'] = r_cme[id_obs_flank].value
            flank.loc[i, 'el'] = np.rad2deg(e_obs[id_obs_flank])

        # Force values to be floats.
        keys = ['lon', 'r', 'el']
        flank[keys] = flank[keys].astype(np.float64)
        return flank
    
    def compute_synthetic_obs(self, el_spread=0.5, cadence=5, el_min=4.0, el_max=30.0):
        """
        Return synthetic observations with a specified uncertainty spread, cadence, and maximum elongation.
        el_spread = standard deviation of random gaussian noise added to the modelled elongation.
        cadence = The cadence with witch observations are returned, as a whole number of model time steps.
        el_min = The minimum elongation of the observers field of view.
        el_max = The maximum elongation of the observers field of view.
        """

        # Compute the time-elongation profiles of the CME flanks from STA and STB
        model_flank = self.model_flank.copy()

        # Remove invalid points
        model_flank.dropna(inplace=True)

        # Add observation noise.
        obs_flank = model_flank.loc[:, ['time', 'el']].copy()
        obs_flank['el'] = obs_flank['el'] + el_spread*np.random.randn(obs_flank.shape[0])

        # Only keep every dt_scale'th observation and reindex - dt_scale=5 corrsponds to ~2hr
        obs_flank = obs_flank[::cadence]
        obs_flank.set_index(np.arange(0, obs_flank.shape[0]), inplace=True)

        # Only return up to el_max ~ (approx HI1 FOV is 25deg)
        id_fov = (obs_flank['el'] >= el_min) & (obs_flank['el'] <= el_max)
        obs_flank = obs_flank[id_fov]
        return obs_flank

def compute_observation_likelihood(t_obs, e_obs, profile):
    """
    Compute the likelihood of an observed elongation measurement for a modelled elongation measurement.
    Assumes a gaussian likelihood centered on the modelled elongation.
    """
    
    # Find the modelled elon at observation time
    e_mod = profile.loc[profile['time'] == t_obs, 'el'].values[0]
    # Compute likelihood of the observation.
    if np.isnan(e_mod):
        lkhd = np.NaN
    else:
        lkhd = st.norm.pdf(e_obs, loc=e_mod, scale=0.5)
        
    return lkhd

def get_cme_params_for_sir(cme):
    """
    Form an array of the CME parameter values that are kept track of in the ensemble members.
    """
    params = np.array([cme.t_launch.to('s').value, cme.longitude.to('rad').value, cme.latitude.to('rad').value,
                          cme.width.to('rad').value, cme.v.value, cme.thickness.to('km').value])
    
    return params

def create_analysis_output_file(filename):
    """
    Create a HDF5 file for storing the SIR analysis steps.
    """
    proj_dirs = H._setup_dirs_()
    out_filepath = os.path.join(proj_dirs['out_data'], filename)
    if os.path.isfile(out_filepath):
        # File exists, so delete and start new.
        print("Warning: {} already exists. Overwriting".format(out_filepath))
        os.remove(out_filepath)

    out_file = h5py.File(out_filepath, 'w')
    return out_file

def cme_kde_resample_with_weights(cme_prior, weights):
    """
    Use kernel density estimation to resample particles from the prior distriubtion given the particle weights.
    """
    # Find valid weights, and pull out each corresponding particles parameters
    n_ensemble = len(weights)
    valid_weights = np.isfinite(weights)
    weights = weights[valid_weights]
    lon = cme_prior[valid_weights, 1].squeeze()
    lat = cme_prior[valid_weights, 2].squeeze()
    width = cme_prior[valid_weights, 3].squeeze()
    v = cme_prior[valid_weights, 4].squeeze()
    thick = cme_prior[valid_weights, 5].squeeze()
    
    # Force lon so no disconinuity at 2pi
    lon[lon>=np.pi] += -2*np.pi

    # KDE fit each distribution, draw random sample. 
    cme_update = np.zeros(cme_prior.shape)*np.NaN
    cme_update[:, 0] = cme_prior[:, 0].copy()
    
    col_id = [1,2,3,4,5]
    param = [lon, lat, width, v, thick]
    for col, p in zip(col_id, param):
        kde = st.gaussian_kde(p, bw_method='silverman',  weights=weights)
        sample = kde.resample(n_ensemble)
        cme_update[:, col] = sample.copy()
        
    # Put lon back on 0-2pi domain
    id_low = cme_update[:, 1] < 0
    cme_update[id_low, 1] += 2*np.pi
    
    return cme_update


def update_ensemble_conecmes(cme_params):
    """
    Produce the list of updated conecmes.
    """
    conecme_update = []
    for i in range(cme_params.shape[0]):
        conecme = H.ConeCME(t_launch=cme_params[i, 0]*u.s,
                            longitude=cme_params[i, 1]*u.rad,
                            latitude=cme_params[i, 2]*u.rad,
                            width=cme_params[i, 3]*u.rad,
                            v=cme_params[i, 4]*(u.km/u.s),
                            thickness=(cme_params[i, 5]*u.km).to(u.solRad))
        conecme_update.append(conecme)
        
    return conecme_update


def plot_huxt_with_observer(time, model, observer, add_flank=False, add_fov=False):
    
    id_t = np.argmin(np.abs(model.time_out - time))

    # Get plotting data
    lon_arr, dlon, nlon = H.longitude_grid()
    lon, rad = np.meshgrid(lon_arr.value, model.r.value)
    mymap = mpl.cm.viridis
    v_sub = model.v_grid_cme.value[id_t, :, :].copy()
    # Insert into full array
    if lon_arr.size != model.lon.size:
        v = np.zeros((model.nr, nlon)) * np.NaN
        if model.lon.size != 1:
            for i, lo in enumerate(model.lon):
                id_match = np.argwhere(lon_arr == lo)[0][0]
                v[:, id_match] = v_sub[:, i]
        else:
            print('Warning: Trying to contour single radial solution will fail.')
    else:
        v = v_sub

    # Pad out to fill the full 2pi of contouring
    pad = lon[:, 0].reshape((lon.shape[0], 1)) + model.twopi
    lon = np.concatenate((lon, pad), axis=1)
    pad = rad[:, 0].reshape((rad.shape[0], 1))
    rad = np.concatenate((rad, pad), axis=1)
    pad = v[:, 0].reshape((v.shape[0], 1))
    v = np.concatenate((v, pad), axis=1)

    mymap.set_over('lightgrey')
    mymap.set_under([0, 0, 0])
    levels = np.arange(200, 800 + 10, 10)
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={"projection": "polar"})
    cnt = ax.contourf(lon, rad, v, levels=levels, cmap=mymap, extend='both')

    # Add on CME boundaries and Observer
    cme = model.cmes[0]
    ax.plot(cme.coords[id_t]['lon'], cme.coords[id_t]['r'], '-', color='darkorange', linewidth=3, zorder=3)
    ert = model.get_observer('EARTH')
    ax.plot(ert.lon[id_t], ert.r[id_t], 'co', markersize=16, label='Earth')            

    # Add on the observer
    ax.plot(observer.lon[id_t], observer.r[id_t], 's', color='r', markersize=16, label='Observer')
        
    if add_flank:
        flank_lon = observer.model_flank.loc[id_t,'lon']
        flank_rad = observer.model_flank.loc[id_t,'r']
        ax.plot(flank_lon, flank_rad, 'r.', markersize=10, zorder=4)
        # Add observer-flank line
        ro = observer.r[id_t]
        lo = observer.lon[id_t]
        ax.plot([lo.value, flank_lon], [ro.value, flank_rad], 'r--', zorder=4)
        
    if add_fov:
        flank_lon = observer.model_flank.loc[id_t,'lon']
        flank_rad = observer.model_flank.loc[id_t,'r']
        fov_patch = get_fov_patch(observer.r[id_t], observer.lon[id_t], observer.el_min, observer.el_max)
        ax.add_patch(fov_patch)

    ax.set_ylim(0, 240)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.patch.set_facecolor('slategrey')

    fig.subplots_adjust(left=0.05, bottom=0.16, right=0.95, top=0.99)
    # Add color bar
    pos = ax.get_position()
    dw = 0.005
    dh = 0.045
    left = pos.x0 + dw
    bottom = pos.y0 - dh
    wid = pos.width - 2 * dw
    cbaxes = fig.add_axes([left, bottom, wid, 0.03])
    cbar1 = fig.colorbar(cnt, cax=cbaxes, orientation='horizontal')
    cbar1.set_label('Solar Wind speed (km/s)')
    cbar1.set_ticks(np.arange(200, 810, 100))
    return fig, ax

def get_fov_patch(ro, lo, el_min, el_max):
    """
    Function to compute a matplotlib patch to higlight an observers field of view. 
    ro = radius of observer (in solRad)
    lo = longitude of observer (in rad)
    el_min = minimum elongation of the field of view
    el_max = maximum elongation of the field of view
    """
    xo = ro*np.cos(lo)
    yo = ro*np.sin(lo)
    
    fov_patch =[[lo.value, ro.value]]
    
    for el in [el_min, el_max]:

        rp = ro*np.tan(el*u.deg)
        if (lo < 0*u.rad) | (lo > np.pi*u.rad):
            lp = lo + 90*u.deg
        else:
            lp = lo - 90*u.deg

        if lp > 2*np.pi*u.rad:
            lp = lp - 2*np.pi*u.rad

        xp = rp*np.cos(lp)
        yp = rp*np.sin(lp)

        # Wolfram equations for intersection of line with circle
        rf = 475*u.solRad # set this to a large value outside axis lims so FOV shading spans model domain 
        dx = (xp - xo)
        dy = (yp - yo)
        dr = np.sqrt(dx**2 + dy**2)
        D = (xo*yp - xp*yo)
        discrim = np.sqrt((rf*dr)**2 - D**2)

        if (lo < 0*u.rad) | (lo > np.pi*u.rad) :
            xf = (D*dy + np.sign(dy)*dx*discrim) / (dr**2)
            yf = (-D*dx + np.abs(dy)*discrim) / (dr**2)
        else:
            xf = (D*dy - np.sign(dy)*dx*discrim) / (dr**2)   
            yf = (-D*dx - np.abs(dy)*discrim) / (dr**2)

        lf = np.arctan2(yf, xf)
        fov_patch.append([lf.value, rf.value])

    fov_patch = mpl.patches.Polygon(np.array(fov_patch), color='r', alpha=0.3, zorder=1)
    return fov_patch


def animate_observer(model, obs, tag, add_flank=False, add_fov=False):
    """
    Animate the model solution, and save as an MP4.
    :param field: String, either 'cme', or 'ambient', specifying which solution to animate.
    :param tag: String to append to the filename of the animation.
    """
    # Set the duration of the movie
    # Scaled so a 5 day simulation with dt_scale=4 is a 10 second movie.
    duration = model.simtime.value * (10 / 432000)

    def make_frame(t):
        """
        Produce the frame required by MoviePy.VideoClip.
        :param t: time through the movie
        """
        # Get the time index closest to this fraction of movie duration
        i = np.int32((model.nt_out - 1) * t / duration)
        fig, ax = plot_huxt_with_observer(model.time_out[i], model, obs, add_flank=add_flank, add_fov=add_fov)
        frame = mplfig_to_npimage(fig)
        plt.close('all')
        return frame

    cr_num = np.int32(model.cr_num.value)
    filename = "HUXt_CR{:03d}_{}_movie.mp4".format(cr_num, tag)
    filepath = os.path.join(model._figure_dir_, filename)
    animation = mpy.VideoClip(make_frame, duration=duration)
    animation.write_videofile(filepath, fps=24, codec='libx264')
    return

def compute_profile_difference(obs_flank, model_flank):
    """
    Compute the rms difference between the observed and modelled flank elongations. 
    """
    # Do interp rather than join of dataframes, as it generalises out of model world. 
    elon_interp = np.interp(obs_flank['time'].values, model_flank['time'].values, model_flank['el'].values, left=np.NaN, right=np.NaN)
    matched_model = pd.DataFrame({'time': obs_flank['time'].values, 'el': elon_interp})
    de = (matched_model['el'] - obs_flank['el'])**2
    n_rms_samp = np.sum(np.isfinite(de))
    rms = np.sqrt(de.mean(skipna=True))
    return rms, n_rms_samp

In [3]:
np.random.seed(20200114)
start_time = Time('2008-01-01T00:00:00')
model = setup_huxt(start_time, uniform_wind=True)

# Initialise Earth directed CME. Coords in HEEQ, so need Earth Lat.
ert = model.get_observer('EARTH')
avg_ert_lat = np.mean(ert.lat.to(u.deg).value)
cme_base = get_base_cme(v=1000, lon=0, lat=avg_ert_lat, width=35)

# Perturb the base CME to get a "pseudo-truth", and solve
cme_truth = perturb_cone_cme(cme_base)
model.solve([cme_truth])

cme_truth = model.cmes[0]
observer_lon = -60*u.deg
observer = Observer(model, observer_lon, el_min=4.0, el_max=40.0)
true_observed_flank = observer.compute_synthetic_obs(el_spread=0.5, cadence=3, el_min=observer.el_min, el_max=observer.el_max)
#animate_observer(model, observer, 'observer_test', add_flank=True, add_fov=True)
    
# Now produce ensemble run.
n_ensembles = 20
n_members = 50

filename = "SIR_HUXt_test.hdf5"
out_file = h5py.File(filename, 'w')

out_file.create_dataset('n_ensembles', data=n_ensembles)

cme_params = get_cme_params_for_sir(cme_base)
out_file.create_dataset('base_cme_params', data=cme_params)
cme_params = get_cme_params_for_sir(cme_truth)
out_file.create_dataset('truth_cme_params', data=cme_params)
out_file.create_dataset('true_model_flank', data=observer.model_flank.values)
out_file.create_dataset('true_observed_flank', data=true_observed_flank.values)

for i in range(n_ensembles):
    
    ensgrp = out_file.create_group('ensemble_{:02d}'.format(i))
    ensgrp.create_dataset('n_members', data=n_members)
    
    for j in range(n_members):
    
        # Perturb the CME and solve
        cme_ens = perturb_cone_cme(cme_base)
        model.solve([cme_ens])
        cme_ens = model.cmes[0]

        ens_observer = Observer(model, observer_lon, el_min=4.0, el_max=40.0)

        # Compute RMSE between ens_observer model flank and truth observations
        profile_rms, n_samp = compute_profile_difference(true_observed_flank, ens_observer.model_flank)

        # Stash this CMEs parameters, observed flank, and arrival time
        memgrp = ensgrp.create_group('member_{:02d}'.format(j))
        cme_params = get_cme_params_for_sir(cme_ens)
        memgrp.create_dataset('cme_params', data=cme_params)
        memgrp.create_dataset('model_flank', data=ens_observer.model_flank.values)
        memgrp.create_dataset('arrival', data=cme_ens.earth_arrival_time.jd)
        memgrp.create_dataset('profile_rms', data=profile_rms)
        memgrp.create_dataset('n_rms_samples', data=n_samp)
        
    out_file.flush()

out_file.close()    

Files already exist for CR2065


KeyboardInterrupt: 

In [15]:
def plot_arrival_dist(i, arrivals, weights, arrival_tru):

    arrivals = Time(np.array(arrivals),format='jd')

    arrival_tru = Time(arrival_tru, format='jd')
    window = 5.0/24
    dt = 1.0/24
    bins = np.arange((arrival_tru - window).jd, (arrival_tru + window+dt).jd, dt)
    bins = Time(bins, format='jd')

    fig, ax = plt.subplots(figsize=(7,7))
    ax.hist(arrivals.datetime, bins.datetime, density=True, color='lightgrey', label='Prior', alpha=1.0)
    ax.hist(arrivals.datetime, bins.datetime, density=True, weights=weights, color='deepskyblue', alpha=0.5, label='Posterior')
    ax.vlines(arrival_tru.datetime, 0,10,'r')

    ax.set_xlabel('CME arrival time (day-hour)')
    hours = mpl.dates.HourLocator(interval=4)   # every hour
    hours_fmt = mpl.dates.DateFormatter('%d-%H')
    # format the ticks
    ax.xaxis.set_major_locator(hours)
    ax.xaxis.set_major_formatter(hours_fmt)

    ax.set_ylabel('Frequency')
    ax.legend()
    figname = "arrival_hist_ensemble_{:02d}.png".format(i)
    fig.savefig(figname)
    plt.close('all')
    return

In [31]:
# Collect the arrival time estimates, and compute the rmse of each profile
filename = "SIR_HUXt_test.hdf5"
out_file = h5py.File(filename, 'r')

arrival_tru_jd = cme_truth.earth_arrival_time.jd

n_ensembles = 48 # bodge as quit early
avg_arrival_prior = np.zeros(n_ensembles)
avg_arrival_posterior = np.zeros(n_ensembles)

for i in range(n_ensembles):
    ensemble_key = 'ensemble_{:02d}'.format(i)
    ensemble = out_file[ensemble_key]
    arrivals = np.zeros(n_members)
    profile_rmse = np.zeros(n_members)
    for j in range(n_members):
    
        member_key = 'member_{:02d}'.format(j)
        arrivals[j] = ensemble[member_key+"/arrival"][()]
        profile_rmse[j] = ensemble[member_key+"/profile_rms"][()]
        
    
    avg_arrival_prior[i] = np.average(arrivals)
    weights = 1.0 / profile_rmse
    weights = weights / np.sum(weights)
    avg_arrival_posterior[i] = np.average(arrivals, weights=weights)
    #plot_arrival_dist(i, arrivals, weights, arrival_tru_jd)

out_file.close()

[2454468.83852218 2454468.83314612 2454468.84263403 2454468.85218885
 2454468.84477214 2454468.83711563 2454468.83639239 2454468.8445641
 2454468.8413798  2454468.8375605  2454468.83501119 2454468.826581
 2454468.84519123 2454468.85548864 2454468.85847644 2454468.85094247
 2454468.84416314 2454468.83135721 2454468.8455644  2454468.8392982
 2454468.82752657 2454468.83920889 2454468.84326878 2454468.83836525
 2454468.84142702 2454468.84310046 2454468.8494239  2454468.84470091
 2454468.85207925 2454468.8293577  2454468.83526597 2454468.83274438
 2454468.83462817 2454468.84251292 2454468.84140724 2454468.84645476
 2454468.83893983 2454468.83291845 2454468.83441399 2454468.82944362
 2454468.83589208 2454468.84552084 2454468.85513425 2454468.83693584
 2454468.84070317 2454468.84660943 2454468.84307534 2454468.84013807]
[2454468.84690222 2454468.84794539 2454468.8547109  2454468.86388313
 2454468.85913654 2454468.85221058 2454468.84849967 2454468.85426737
 2454468.85177705 2454468.84780535 24

In [73]:
# Collect the arrival time estimates, and compute the rmse of each profile
filename = "SIR_HUXt_test.hdf5"
out_file = h5py.File(filename, 'r')

arrival_tru_jd = cme_truth.earth_arrival_time.jd

n_ensembles = 48 # bodge as quit early
avg_arrival_prior = np.zeros(n_ensembles)
avg_arrival_posterior = np.zeros(n_ensembles)
v_prior = np.zeros(n_ensembles)
v_posterior = np.zeros(n_ensembles)

for i in range(n_ensembles):
    ensemble_key = 'ensemble_{:02d}'.format(i)
    ensemble = out_file[ensemble_key]
    arrivals = np.zeros(n_members)
    profile_rmse = np.zeros(n_members)
    cme_params = np.zeros((n_members, 6))
    for j in range(n_members):
    
        member_key = 'member_{:02d}'.format(j)
        arrivals[j] = ensemble[member_key+"/arrival"][()]
        profile_rmse[j] = ensemble[member_key+"/profile_rms"][()]
        cme_params[j, :] = ensemble[member_key+"/cme_params"][()]
    
    avg_arrival_prior[i] = np.average(arrivals)
    weights = 1.0 / profile_rmse
    weights = weights / np.sum(weights)
    avg_arrival_posterior[i] = np.average(arrivals, weights=weights)
    
    v_prior[i] = np.average(cme_params[:, 4])
    v_posterior[i] = np.average(cme_params[:, 4], axis=0, weights=weights)
    

out_file.close()

In [77]:
err_v_pri = v_prior - cme_truth.v.value
err_v_pos = v_posterior - cme_truth.v.value

rmse_v_pri = np.sqrt(np.mean(err_v_pri**2))
rmse_v_pos = np.sqrt(np.mean(err_v_pos**2))

v_pri_avg = np.mean(v_prior)
#v_pri_sem = 2*st.sem(v_prior)
v_pri_sem = 2*np.std(v_prior)

v_pos_avg = np.mean(v_posterior)
#v_pos_sem = 2*st.sem(v_posterior)
v_pos_sem = 2*np.std(v_posterior)

print("Truth:{}   Prior:{}+/-{}   Posterior:{}+/-{}".format(cme_truth.v, v_pri_avg, v_pri_sem, v_pos_avg, v_pos_sem))

print(rmse_v_pri, rmse_v_pos)

Truth:942.5513668409623 km / s   Prior:999.7053023963266+/-14.452147857208866   Posterior:988.0205062801219+/-15.242377327499844
57.60892720653977 46.10341807270051


In [34]:
err_prior = (avg_arrival_prior - arrival_tru_jd)*24
err_posterior = (avg_arrival_posterior - arrival_tru_jd)*24

rmse_prior = np.sqrt(np.mean(err_prior**2))
rmse_posterior = np.sqrt(np.mean(err_posterior**2))

print(rmse_prior, rmse_posterior, rmse_prior/rmse_posterior)

2.510185951282149 2.23616169252759 1.1225422381888772
